In [1]:
import tensorflow as tf
import random
# import matplotlib.pyplot as plt

import csv

fr = open('th4_train.csv', 'r')
fr.readline()
read_data = csv.reader(fr)

In [2]:
mnist_data = list(read_data)

In [3]:
#print(mnist_data)

In [4]:
print( len(mnist_data) )

16877


In [5]:
fr.close()

train_image = []
train_label = []

##############from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)  # reproducibility

##############mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset

# hyper parameters
learning_rate = 0.1
training_epochs = 1
batch_size = 900


In [6]:
optimizer_func=tf.train.AdamOptimizer
act_func=tf.nn.relu


In [7]:
def get_train_batch(batch_size):
    
    batch_image = []
    batch_label = []
    random.shuffle(mnist_data)
    
    for i in range(batch_size):
        each_image = mnist_data[i][1:9]
        label1 = mnist_data[i][9]
        label2 = mnist_data[i][10]
        label3 = mnist_data[i][11]
        #for j in range(len(each_image)):
        #    each_image[j] = ((float)(each_image[j]) / 255)
        
        batch_image.append(each_image)
        batch_label.append([label1, label2, label3])
        """if mnist_data[i][0] == '0':
            batch_label.append([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        """
    #print(batch_image[0:batch_size], '     ',batch_label[0:batch_size])
    return batch_image[0:batch_size], batch_label[0:batch_size] 

In [8]:
with tf.name_scope('input') as scope:
    X = tf.placeholder(tf.float32, [None, 8], name = 'image')
    y = tf.placeholder(tf.float32, [None, 3], name = 'label')    

W = []
b = []
L = []

for i in range(0, 151):
    W.append([])
    b.append([])
    L.append([])

with tf.variable_scope('layer1') as scope:
    W[1] = tf.get_variable("W", shape = [8, 8], initializer = tf.contrib.layers.xavier_initializer())
    b[1] = tf.Variable(tf.random_normal([8]))
    L[1] = act_func(tf.add(tf.matmul(X, W[1]), b[1]))#########################################ex1-2act_func
    
    tf.summary.histogram("X", X)
    tf.summary.histogram("weights", W[1])
    tf.summary.histogram("bias", b[1])
    tf.summary.histogram("layer", L[1]) 
    

    
for i in range(2, 151):
    layer_name = 'layer' + str(i)    
    with tf.variable_scope(layer_name) as scope:
        W[i] = tf.get_variable("W", shape = [8, 8], initializer = tf.contrib.layers.xavier_initializer())
        b[i] = tf.Variable(tf.random_normal([8]))
        L[i] = act_func(tf.add(tf.matmul(L[i-1], W[i]), b[i]))#########################################ex1-2act_func
        
        tf.summary.histogram("weights", W[i])
        tf.summary.histogram("bias", b[i])
        tf.summary.histogram("layer", L[i])

In [9]:
print('wow ',i)
print(L[150])

wow  150
Tensor("layer150/Relu:0", shape=(?, 8), dtype=float32)


In [10]:
with tf.variable_scope('layer_out') as scope:
    W0 = tf.get_variable("W", shape = [8, 3], initializer = tf.contrib.layers.xavier_initializer())
    b0 = tf.Variable(tf.random_normal([3]))
    y_ = tf.add(tf.matmul(L[150], W0), b0)
    
    tf.summary.histogram("weights", W0)
    tf.summary.histogram("bias", b0)
    tf.summary.histogram("logits", y_)

In [11]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_, labels = y))
optimizer = optimizer_func(learning_rate = learning_rate).minimize(loss) #################################ex1-2optimizer
tf.summary.scalar("loss", loss)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



<tf.Tensor 'loss:0' shape=() dtype=string>

In [12]:
summary = tf.summary.merge_all()

In [13]:
print(len(mnist_data))

16877


In [14]:
def read_data(name):
    fr = open(name,'r')
    fr.readline()
    read_data = csv.reader(fr)
    ret_data = list(read_data)
    fr.close()
    return ret_data   

In [15]:
#forecast
fc_dic = []
fc_add = read_data('forecast_london_me_20180523.csv')

#dictionary에 aq데이터를 입력
for i in range(len(fc_add)):    
    
    if fc_add[i][1] == 'london_grid_430':            
        fc_dic.append(fc_add[i])

In [21]:
global_step = 0
with tf.Session() as sess:

    with tf.device("/gpu:0"):
        sess.run(tf.global_variables_initializer())
    
        for epoch in range(training_epochs):
            total_batch = int( len(mnist_data) / batch_size )
            avg_loss = 0
        
            for i in range(total_batch):
                batch_xs, batch_ys = get_train_batch(batch_size)
                feed_dict = {X: batch_xs, y: batch_ys}
                s, I, _ = sess.run([summary, loss, optimizer], feed_dict = feed_dict)
                #print('s is ',s,'   I is',I)
                global_step += 1
                avg_loss += I
            print('Epoch:', '%02d' % (epoch + 1), 'loss = ', '{:6f}'.format(avg_loss / total_batch))
            #print('s is ',s,'   I is',I)
    
        correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        for i in range(len(mnist_data)):    
            if mnist_data[i][0] == 'TH4#1297':
                #print(len(mnist_data[i]))
                pm25 = mnist_data[i][9]
                pm10 = mnist_data[i][10]
                o3 = mnist_data[i][11]     
                test_data = []
                test_data.append([pm25, pm10, o3, mnist_data[i][4], mnist_data[i][5], mnist_data[i][6], mnist_data[i][8], mnist_data[i][7] ])
                break             
                #tf.add(tf.matmul(L[150], W0), b0)
        result = []
        output = sess.run(tf.add(tf.matmul(L[150], W0), b0), feed_dict = {X: test_data})
        pm25 = output[0][0]
        pm10 = output[0][1]
        o3 = output[0][2]
        result.append([pm25, pm10, o3])
        
        for i in range(47):
            print(test_data)
            test_data[0][0] = pm25
            test_data[0][1] = pm10
            test_data[0][2] = o3
            
            if fc_dic[i][4] != '':
                test_data[0][3] = fc_dic[i][4]
            if fc_dic[i][5] != '':
                test_data[0][4] = fc_dic[i][5]
            if fc_dic[i][6] != '':
                test_data[0][5] = fc_dic[i][6]
            if fc_dic[i][8] != '':
                test_data[0][6] = fc_dic[i][8]
            if fc_dic[i][7] != '':
                test_data[0][7] = fc_dic[i][7]
            
            output = sess.run(tf.add(tf.matmul(L[150], W0), b0), feed_dict = {X: test_data})
            
            pm25 = output[0][0]
            pm10 = output[0][1]
            o3 = output[0][2]
            result.append([pm25, pm10, o3])
            
        
        #print(len(output))
        #print(output)


Epoch: 01 loss =  113.754999
[['17.1', '26.9', '32.3', '16.08', '1014.5234', '76.0', '9.27', '12.7']]
[[0.3504181, 2.341681, 1.4781897, '12.62', '1017.48411373436', '79.0', '38.32', '14.77']]
[[0.3504181, 2.341681, 1.4781897, '12.73', '1017.16813019273', '82.0', '40.98', '14.46']]
[[0.3504181, 2.341681, 1.4781897, '12.94', '1016.911171875', '84.0', '44.05', '14.3']]
[[0.3504181, 2.341681, 1.4781897, '13.24', '1016.7227993083', '84.0', '47.86', '14.22']]
[[0.3504181, 2.341681, 1.4781897, '13.62', '1016.61870012883', '83.0', '51.6', '13.99']]
[[0.3504181, 2.341681, 1.4781897, '14.04', '1016.61609375', '82.0', '54.35', '13.28']]
[[0.3504181, 2.341681, 1.4781897, '14.54', '1016.70468324538', '82.0', '55.67', '11.92']]
[[0.3504181, 2.341681, 1.4781897, '15.32', '1016.76410632901', '82.0', '57.06', '10.58']]
[[0.3504181, 2.341681, 1.4781897, '16.64', '1016.646484375', '79.0', '60.85', '10.18']]
[[0.3504181, 2.341681, 1.4781897, '18.59', '1016.25643935367', '71.0', '67.16', '11.41']]
[[0.3504

In [19]:
 print(output[0][2])       
 

1.4758003


In [20]:
csvfile = open('th4TEST', "w", newline="")#csvfile = open("./submit.csv", "w", newline="")
    
csvwriter = csv.writer(csvfile)
csvwriter.writerow(['test_id','PM2.5','PM10','O3'])


        

for j in range( len( result ) ):
    i = 'TH4'
    st_t = i + '#' + str(j)
    csvwriter.writerow([st_t, result[j][0], result[j][1], result[j][2]])


csvfile.close()